In [70]:
import numpy as np
from lightfm.data import Dataset
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score
from lightfm.cross_validation import random_train_test_split
from scipy.sparse import coo_matrix as sp
import sys

In [26]:
import pandas as pd
user_prod=pd.read_csv("_User_Prod_.csv")
user_features=pd.read_csv("_User_Features_.csv")
prod_features=pd.read_csv("_Prod_Features_.csv")

In [27]:
user_prod.head()

,hyb_dat_correo,id_producto,unidades_totales
0,m.ureta.leon@gmail.com,200091530,1
1,tivisay88@hotmail.com,200060404,1
2,amzoason@gmail.com,P0210111001,3
3,asanchezc54@gmail.com,200081874,1
4,maria15aguilera@live.com,200051086,1


In [ ]:
prod_features.head()

In [29]:
user_features.head()

,hyb_dat_correo,hyb_codpais,grupo_frecuencia,grupo_ticket,grupo_recencia
0,nidia.rojas40506@gmail.com,CO,F3,M4,R1
1,juanisrocio@hotmail.com,CO,F3,M3,R1
2,mbriverah@yahoo.com,PE,F2,M1,R3
3,y_torrenegra@hotmail.com,CO,F2,M2,R3
4,ervillac14@gmail.com,CO,F3,M3,R1


In [30]:
user_prod_interaction = pd.pivot_table(user_prod, index='hyb_dat_correo', columns='id_producto', values='unidades_totales')# fill missing values with 0
user_prod_interaction = user_prod_interaction.fillna(0)

In [31]:
user_prod_interaction

id_producto,200004616,200007423,200009812,200010080,200011767,200011773,200012014,200012707,200012761,200014172,...,P0292141004,P0292141009,P0297050000,P0297050001,P0297050002,P0297050004,P0297050008,P0297051000,P0406051000,P0454050000
hyb_dat_correo,,,,,,,,,,,,,,,,,,,,,
000314282@adexperu.edu.pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0125@live.com.mx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
014200549a@uandina.edu.pe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
015200818a@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
017.karen@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zxuxa@hotmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyan.sm@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zyanya2807@hotmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
user_id = list(user_prod_interaction.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [33]:
# convert to csr matrix
from scipy.sparse import csr_matrix
user_prod_interaction_csr = csr_matrix(user_prod_interaction.values)
user_prod_interaction_csr

<51367x1741 sparse matrix of type '<class 'numpy.float64'>'
	with 196741 stored elements in Compressed Sparse Row format>

In [34]:
item_dict ={}
df = prod_features[['id_producto', 'nom_producto']].sort_values('id_producto').reset_index()
for i in range(df.shape[0]):
    item_dict[(df.loc[i,'id_producto'])] = df.loc[i,'nom_producto']
    

In [35]:
# dummify categorical features - product
prod_metadata_selected_transformed = pd.get_dummies(prod_features, columns = ['desmarca', 'descategoria', 'dessubcategoriasolo','rango_precio'])
prod_metadata_selected_transformed = prod_metadata_selected_transformed.sort_values('id_producto').reset_index().drop('index', axis=1)
prod_metadata_selected_transformed.head(10)



,id_producto,nom_producto,desmarca_CYZONE,desmarca_ESIKA,desmarca_GENERICA,desmarca_LBEL,descategoria_ACCESORIOS COSMETICOS,descategoria_CABALLEROS,descategoria_COMPLEMENTOS,descategoria_CUIDADO PERSONAL,...,dessubcategoriasolo_TRATAMIENTO FACIAL,dessubcategoriasolo_UNISEX,dessubcategoriasolo_VARIOS,rango_precio_M10,rango_precio_M15,rango_precio_M20,rango_precio_M25,rango_precio_M30,rango_precio_M35,rango_precio_M5
0,200004616,LB DERMOCLARITE AM 30ML,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,200004616,LB DERMO CLARITE AM 30ML,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,200007423,LBEL BROCHA RUBOR,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,200009812,ES EXUS 100ML,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,200010080,ES FIORI 50ML,0,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
5,200011767,ES IT'S YOU 50ML,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
6,200011773,ES IT'S YOU 100ML,0,1,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
7,200012014,ES KALOS SPORT 100ML,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,200012707,LB LIASSON 50ML,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9,200012761,LB LIASSON MINI CC2018 10ML,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
# convert to csr matrix
prod_metadata_csr = csr_matrix(prod_metadata_selected_transformed.drop(['id_producto','nom_producto'], axis=1).values)

In [37]:
prod_metadata_csr

<1949x68 sparse matrix of type '<class 'numpy.uint8'>'
	with 7796 stored elements in Compressed Sparse Row format>

In [38]:
# dummify categorical features - user
user_metadata_selected_transformed = pd.get_dummies(user_features, columns = ['grupo_recencia','grupo_ticket','grupo_frecuencia','hyb_codpais'])
user_metadata_selected_transformed = user_metadata_selected_transformed.sort_values('hyb_dat_correo').reset_index().drop('index', axis=1)
user_metadata_selected_transformed.head(20)

,hyb_dat_correo,grupo_recencia_R1,grupo_recencia_R2,grupo_recencia_R3,grupo_ticket_M1,grupo_ticket_M2,grupo_ticket_M3,grupo_ticket_M4,grupo_frecuencia_F1,grupo_frecuencia_F2,grupo_frecuencia_F3,hyb_codpais_CL,hyb_codpais_CO,hyb_codpais_MX,hyb_codpais_PE
0,000314282@adexperu.edu.pe,1,0,0,0,0,0,1,1,0,0,0,0,0,1
1,0125@live.com.mx,0,1,0,0,0,0,1,1,0,0,0,0,1,0
2,014200549a@uandina.edu.pe,0,0,1,0,0,1,0,1,0,0,0,0,0,1
3,015200818a@gmail.com,0,1,0,0,0,0,1,1,0,0,0,0,0,1
4,017.karen@gmail.com,0,1,0,1,0,0,0,1,0,0,0,1,0,0
5,01nana0216@gmail.com,1,0,0,1,0,0,0,1,0,0,0,1,0,0
6,0311andre@gmail.com,1,0,0,1,0,0,0,1,0,0,0,1,0,0
7,0412191086@uladech.pe,0,1,0,1,0,0,0,1,0,0,0,0,0,1
8,0425melannie@gmail.com,1,0,0,0,0,0,1,1,0,0,0,0,0,1
9,04elianasilva@gmail.com,0,0,1,0,0,0,1,0,0,1,0,0,0,1


In [39]:
# convert to csr matrix
user_metadata_csr = csr_matrix(user_metadata_selected_transformed.drop(['hyb_dat_correo'], axis=1).values)

In [40]:
user_metadata_csr

<51372x14 sparse matrix of type '<class 'numpy.uint8'>'
	with 205488 stored elements in Compressed Sparse Row format>

In [41]:
model = LightFM(loss='warp',
                random_state=123,
                learning_rate=0.1,
                no_components=20,
                user_alpha=0.0)
model = model.fit(user_prod_interaction_csr,
                  epochs=10,
                  num_threads=1, verbose=False)

In [42]:
def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items), item_features=prod_metadata_csr, 
                                     user_features=user_metadata_csr))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print ("Cliente: " + str(user_id))
        ##print("Ha comprado:")
        ##counter = 1
        ##for i in known_items:
            ##print(str(counter) + '- ' + i)
            ##counter+=1
        print("\n Productos recomendados:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list
        

In [43]:
lista=sample_recommendation_user(model, user_prod_interaction, 'rod.parej@gmail.com', user_dict, item_dict)
lista

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML


['200095928',
 '200076066',
 '200095922',
 '200076001',
 '200100407',
 '200099944',
 '200093352',
 '200093330',
 '200099970',
 '200054552']

In [44]:
i=1
for i in range (1,1000):
    sample_recommendation_user(model, user_prod_interaction, 'rod.parej@gmail.com', user_dict, item_dict)
    i=i+1

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

...

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
Cliente: rod.parej@gmail.com

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION 

In [45]:
#Tabla de Output
#Puntos de automatización:
#1. Modelo
#2. API actualiza datos del modelo
#3. Job para volver a generar escenario
#4. Enviar resultado de SAP MKT a SAP Commerce
#llave: email 
#forma de carga
#productos a recomendar
#CUC a CodSAP

In [46]:
import pandas as pd
import psycopg2
import base64
connRS=psycopg2.connect(dbname= 'analitico', host='10.12.2.26', 
port= '5439', user= 'rpareja', password= 'B3lcorp2020')

 

result = pd.read_sql("""
DROP TABLE IF EXISTS #T1;
SELECT
cuc as id_producto,
hyb_codsap,
COUNT (DISTINCT hyb_cod_orden) as cant_ordenes
INTO #T1
FROM dom_customer.ecm_res_orden
 WHERE sap_codmarket=''
 AND sap_fechafactura >='2020-07-19 00:00:00' -- Un año atrás
 AND sap_fechafactura <='2020-08-19 00:00:00' -- Hoy
 AND sap_flagfacturaanulada='N'
 AND flg_orden='S'
 AND sap_tipodocumento='M'
 AND sap_flagempaque='N'
GROUP BY 1,2;

DROP TABLE IF EXISTS #T2;
SELECT 
a.id_producto,
MAX(a.cant_ordenes) as cant_ord
INTO #T2
FROM #T1 a
GROUP BY 1;

DROP TABLE IF EXISTS #T3;
SELECT 
a.id_producto,
a.cant_ord,
b.hyb_codsap
INTO #T3
FROM #T2 a
LEFT JOIN #T1 b 
ON a.id_producto=b.id_producto
AND a.cant_ord=b.cant_ordenes;

SELECT * FROM #T3
"""
, connRS)

 

df_sap_representativo = pd.DataFrame(result)

 

connRS.commit()
connRS.close()

 

print(df_sap_representativo.shape)
df_sap_representativo.head(3)

OperationalError: could not connect to server: Connection timed out (0x0000274C/10060)
	Is the server running on host "10.12.2.26" and accepting
	TCP/IP connections on port 5439?


In [ ]:
for rownumber, row in df_sap_representativo.iterrows():
    print(df_sap_representativo.iloc[rownumber])
   

In [47]:
columna_correo=pd.DataFrame(user_prod.hyb_dat_correo.unique())
columna_correo.columns=['id_cliente']
columna_correo

,id_cliente
0,m.ureta.leon@gmail.com
1,tivisay88@hotmail.com
2,amzoason@gmail.com
3,asanchezc54@gmail.com
4,maria15aguilera@live.com
...,...
51363,saraynikol@hotmail.com
51364,kalexa_502@hotmail.com
51365,claudichi2001@hotmail.com
51366,eddypino_21@hotmail.com


In [48]:
numero_columnas=10
for i in range(1,numero_columnas+1):
     columna_correo['producto_'+ str(i)]=''

columna_correo.head()

,id_cliente,producto_1,producto_2,producto_3,producto_4,producto_5,producto_6,producto_7,producto_8,producto_9,producto_10
0,m.ureta.leon@gmail.com,,,,,,,,,,
1,tivisay88@hotmail.com,,,,,,,,,,
2,amzoason@gmail.com,,,,,,,,,,
3,asanchezc54@gmail.com,,,,,,,,,,
4,maria15aguilera@live.com,,,,,,,,,,


In [71]:
i=0 
for rownumber, row in columna_correo.iterrows():
    lista=sample_recommendation_user(model, user_prod_interaction, row['id_cliente'], user_dict, item_dict)
    for j in range(1,numero_columnas+1):
        columna_correo.loc[rownumber,'producto_'+ str(j)]=lista[j-1]
    sys.stdout.write('\r'+str('{:1}'.format(str(rownumber))))

Cliente: m.ureta.leon@gmail.com

 Productos recomendados:
1- CY VICE EST ARE X12
2- CZ XYF LOCION BIFASICA 90ML
3- ES OZEAN HIDROANT 75G
4- LB DREAM MER 50ML
5- ES SET RED & BLACK 3UN
6- ES CFX SET TRENDY COLLECTION
7- ES AGU SH ALMENDRA 1L
8- ES PERF BLOCK R&C FPS50 80ML
9- ES VIVE BERRY LOVE 400 ML
10- ES VIBRANZA LOC PER 150ML
0Cliente: tivisay88@hotmail.com

 Productos recomendados:
1- FI LADY STARK ARE
2- ES NC MYSTIC GRANADA 200ML
3- CZ TRUE 50ML
4- ES NC CARIBBEAN PASSION 200ML
5- CY CARGADOR OH MY LIPS CG
6- CY PORTATODO SEA
7- CY COLGANTE FRONT ROW
8- FI LUCKY ELEPHANT EST C+A
9- LB MATISSE OPTIM MANOS 75G
10- ES VANILLA SCE DESO 50ML
1Cliente: amzoason@gmail.com

 Productos recomendados:
1- ES TAC BBCREAM CLARO 30G
2- ES RENACER SUERO 30ML
3- ES TAC MAX 55+ 120G
4- ES ESENS LISO CREMP 194G
5- ES TAC MAX MASCARILLA 90G
6- ES ESENS PRO SALON AC 350ML
7- LB DERMO CLARITE AM 30ML
8- LB MITHYKA MINI 10ML
9- LB POLVOS DIVINE PLATEADO
10- ES MULT FIRM REFIL 750ML
2Cliente: asanchezc

25Cliente: adst_piscis@hotmail.com

 Productos recomendados:
1- ES ESENS PRO SALON AC 350ML
2- ES TAC MAX MASCARILLA 90G
3- ES MULT NUTRIENTS REFILL 750ML
4- CZ SL LL SET TERRA 2019 3UN
5- ES MEN ACTIVE 200 ML
6- CZ SANDIA SHAKE 200ML
7- LB DEVOS MAGNETIC 100ML
8- LB BLEU INTENSE 100ML
9- LB BLEU INTENSE MINI 10ML
10- ES KALOS SPORT DESO 50ML
26Cliente: evelyn_1490_5@hotmail.com

 Productos recomendados:
1- LB LEDERM DIA 55+ 50ML
2- LB LEDERM OJOS 45+ 15ML
3- CZ SWEET BLACK ROLL ON 50ML
4- ES AGU SH REP MANZANILLA 750ML
5- ES AGU SH ALMENDRA 1L
6- ES OZEAN HIDROANT 75G
7- ES CFX SET TRENDY COLLECTION
8- ES SET RED & BLACK 3UN
9- LB DREAM MER 50ML
10- CZ XYF LOCION BIFASICA 90ML
27Cliente: jessi.alejita@gmail.com

 Productos recomendados:
1- LB SET MINIS
2- CZ AINNARA 50ML
3- LB MATISSE OPTIM MANOS 75G
4- ES VANILLA SCE DESO 50ML
5- LB DREAM MER 50ML
6- ES CFX SET TRENDY COLLECTION
7- ES SET RED & BLACK 3UN
8- ES AGU SH ALMENDRA 1L
9- CZ XYF LOCION BIFASICA 90ML
10- ES OZEAN HIDROANT 75

51Cliente: mariapia020617@gmail.com

 Productos recomendados:
1- FI CRISTAL FEELING EST C+A
2- CZ CY SOM THREE O'CLOCK 2.4G
3- ES PRO BROCHA OJOS 1 UN
4- CZ ESM KHALI MATE 5ML
5- FI NATURE D'OR EST C+A
6- CY ORGANIZADOR TULIP
7- FI SILVER STARDUST EST C+A
8- CY ORGANIZADOR PERFECT ORDER
9- ES PRO ESPONJA MAQUILLAJE 1 UN
10- CY GOLDEN RING SET ANI X3
52Cliente: sheylam.rivadeneyrac@gmail.com

 Productos recomendados:
1- ES VIBRANZA LOC PER 150ML
2- LB POLVOS TRANSLUCENT
3- CZ DANCING NIGHT ROLL ON 50ML
4- FI HALEY ARE
5- LB LEDERM OJOS 35+ 15ML
6- FI LADY STARK ARE
7- LB MATISSE OPTIM MANOS 75G
8- CZ AINNARA 50ML
9- ES VANILLA SCE DESO 50ML
10- LB SET MINIS
53Cliente: nilhy.01@gmail.com

 Productos recomendados:
1- ES LPI SUAVIZANTE 150ML
2- CY BILLETERA CHARLOTTE
3- CZ SKIN FIRST LIMPIADORA 100G
4- CZ BREAK IN 50ML
5- CZ BE SEXY OLEO CONTROL 60ML
6- ES TAC MAX ACLARADORA 145G
7- ES PRO DESMAQ BIFAS 125ML
8- ES VIBRANZA LOC PER 150ML
9- CZ SL PRIMER 20G
10- CZ CY SOLUXION FOCALIZADO 15G

77Cliente: jldlp8@hotmail.com

 Productos recomendados:
1- LB BASE HYDRA ABSOLUTE 30ML
2- ES OXY SUP KIT30+ TSIZE CJ 3UN
3- FI PEARL FOREST ARE
4- FI SHIMMER LINES PUL
5- FI NIÁGARA PUL
6- FI PALAZZO PEN GLAM
7- FI PALAZZO PEN
8- CY BROWNY EST ARE X12
9- FI GEM LAYERS PEN
10- FI FALLING IN LEAF ARG
78Cliente: kimberlycaniunirsanmartin@gmail.com

 Productos recomendados:
1- ES VIBRANZA LOC PER 150ML
2- CY FUN LIFE SET PUL X3
3- FI GEM LAYERS PEN
4- FI GOLD FLY PEN GLAM
5- FI NIÁGARA PUL
6- FI MIX STAR PEN GLAM
7- FI SOFÍA PEN
8- FI SOFÍA PEN GLAM
9- FI PRINCESS BUNNY PEN GLAM
10- FI PALAZZO PEN GLAM
79Cliente: scherezada24@hotmail.com

 Productos recomendados:
1- ES VANILLA SCE DESO 50ML
2- CZ AINNARA 50ML
3- LB SET MINIS
4- LB MATISSE OPTIM MANOS 75G
5- ES VIBRANZA LOC PER 150ML
6- CZ CY MASCARA SECRETLASH
7- LB ESSENTIAL LIMP NG 180ML
8- ES ILUMINAS 50ML
9- ES PARADISSO MAGIC AURO 200ML
10- LB ZAMAK MINI 10ML
80Cliente: alejacap21@gmail.com

 Productos recomendados:
1- ES LPI SUAVIZAN

103Cliente: katiarosales38@gmail.com

 Productos recomendados:
1- ES ESENS LISO CREMP 194G
2- ES RENACER SUERO 30ML
3- ES TAC BBCREAM CLARO 30G
4- ES TAC MAX 55+ 120G
5- LB LEDERM OJOS 45+ 15ML
6- LB LEDERM DIA 55+ 50ML
7- CZ SWEET BLACK ROLL ON 50ML
8- ES AGU SH REP MANZANILLA 750ML
9- ES ESENS PRO SALON AC 350ML
10- ES TAC MAX MASCARILLA 90G
104Cliente: metorresvas@gmail.com

 Productos recomendados:
1- FI CRISTAL FEELING EST C+A
2- CZ CY SOM THREE O'CLOCK 2.4G
3- CY ORGANIZADOR TULIP
4- FI NATURE D'OR EST C+A
5- FI SILVER STARDUST EST C+A
6- CZ ESM KHALI MATE 5ML
7- ES PRO BROCHA OJOS 1 UN
8- CY ORGANIZADOR PERFECT ORDER
9- CY UTILITARIOS MANICUTE
10- CY UNION CRYSTALS EST C+A
105Cliente: villalba-ca@hotmail.com

 Productos recomendados:
1- CZ TRUE 50ML
2- ES NC MYSTIC GRANADA 200ML
3- ES NC CARIBBEAN PASSION 200ML
4- LB SET MINIS
5- ES VANILLA SCE DESO 50ML
6- CZ AINNARA 50ML
7- LB MATISSE OPTIM MANOS 75G
8- CZ MISS SEXY SHINE CR 130ML
9- LB LEDERM NOCHE 35+ 50ML
10- LB ESSENTIAL L

...

259Cliente: kari1003@hotmail.com

 Productos recomendados:
1- ES NC MYSTIC GRANADA 200ML
2- CZ TRUE 50ML
3- ES NC CARIBBEAN PASSION 200ML
4- FI CRISTAL FEELING EST C+A
5- ES OXY SUP KIT30+ TSIZE CJ 3UN
6- LB BASE HYDRA ABSOLUTE 30ML
7- ES MINICHICS COL RUBI 120ML
8- ES MINICHICS COL KELLY 120ML
9- ES ESENS PRO SALON SH 350ML
10- ES OXY SUP OJOS 15ML
260Cliente: yanethbarrero@gmail.com

 Productos recomendados:
1- LB MATISSE OPTIM MANOS 75G
2- ES VANILLA SCE DESO 50ML
3- CZ AINNARA 50ML
4- LB SET MINIS
5- ES MULTI DUALPROT DESO 50ML
6- ES RENACER ADV DIA 45G
7- LB LEDERM NOCHE 35+ 50ML
8- CZ MISS SEXY SHINE CR 130ML
9- ES VIBRANZA 45ML
10- CZ BERRYLICIOUS SEXY 200ML
261Cliente: lacrisospina@hotmail.com

 Productos recomendados:
1- FI NIÁGARA PEN
2- FI SPARK GARDEN PEN
3- FI BOHO SOUL COLL CORTO
4- FI GOLD FLY PEN
5- FI FALLING IN LEAF PEN GLAM
6- FI LUCKY VIBE PEN GLAM
7- FI HELENNE PUL
8- FI MIX STAR PEN
9- FI MY SECRET PEN GLAM
10- FI BELLAGIO PEN GLAM
262Cliente: nancy_e28@hotmail.co

285Cliente: romantica.dm@gmail.com

 Productos recomendados:
1- LB SET MINIS
2- CZ AINNARA 50ML
3- LB MATISSE OPTIM MANOS 75G
4- ES VANILLA SCE DESO 50ML
5- LB DREAM MER 50ML
6- ES CFX SET TRENDY COLLECTION
7- ES SET RED & BLACK 3UN
8- ES AGU SH ALMENDRA 1L
9- CZ XYF LOCION BIFASICA 90ML
10- ES OZEAN HIDROANT 75G
286Cliente: poltergeist.incubo@live.com.mx

 Productos recomendados:
1- ES AGU SH REP MANZANILLA 750ML
2- LB LEDERM DIA 55+ 50ML
3- CZ SWEET BLACK ROLL ON 50ML
4- LB LEDERM OJOS 45+ 15ML
5- ES VIBRANZA LOC PER 150ML
6- ES CFX SET TRENDY COLLECTION
7- LB DREAM MER 50ML
8- ES OZEAN HIDROANT 75G
9- ES SET RED & BLACK 3UN
10- CZ XYF LOCION BIFASICA 90ML
287Cliente: carmonanton@gmail.com

 Productos recomendados:
1- ES VIBRANZA LOC PER 150ML
2- LB SET MINIS
3- ES VANILLA SCE DESO 50ML
4- CZ AINNARA 50ML
5- LB MATISSE OPTIM MANOS 75G
6- ES EXUS 100ML
7- CZ SL LL SET TERRA 2019 3UN
8- LB LIGNE EXP RECONS SH 250ML
9- LB BLEU INTENSE 100ML
10- LB BLEU INTENSE MINI 10ML
288Cliente: lili

311Cliente: fernie.ys.13@hotmail.com

 Productos recomendados:
1- LB BIO RESIST HIDRATANTE 30ML
2- ES PARADISSO MAGIC AURO 200ML
3- LB DRY EFFECT DESO SPRAY 125ML
4- ES ILUMINAS 50ML
5- ES CARDIGAN 90ML
6- LB ESSENTIAL LIMP NG 180ML
7- ES D'ORSAY 90ML
8- LB ZAMAK MINI 10ML
9- LB NEW CODE RED MINI 10ML
10- CZ BIB SEXY GLOW 180ML
312Cliente: mrisconavarrete@hotmail.com

 Productos recomendados:
1- CZ TRUE 50ML
2- ES NC MYSTIC GRANADA 200ML
3- ES NC CARIBBEAN PASSION 200ML
4- LB SET MINIS
5- ES VANILLA SCE DESO 50ML
6- CZ AINNARA 50ML
7- LB MATISSE OPTIM MANOS 75G
8- CZ MISS SEXY SHINE CR 130ML
9- LB LEDERM NOCHE 35+ 50ML
10- LB ESSENTIAL LIMP NS 180ML
313Cliente: dolceramirez83@gmail.com

 Productos recomendados:
1- LB POLVOS SUELTOS
2- LB DELINEADOR COLOR INTENSE
3- LB MASCARA MEGACILS EXTEND 9G
4- LB POLVOS CONTROL BRILLO 9G
5- LB LEDERM DIA 55+ 50ML
6- LB LEDERM OJOS 45+ 15ML
7- CZ SWEET BLACK ROLL ON 50ML
8- ES AGU SH REP MANZANILLA 750ML
9- LB CORRECTOR FACIAL 1.5ML
10- LB POLVOS DI

337Cliente: marcela.candela.r@gmail.com

 Productos recomendados:
1- LB LEDERM OJOS 35+ 15ML
2- FI HALEY ARE
3- ES TAC 55+ 200G
4- LB L'EGARD HOMME 80ML
5- CZ MISS SEXY 200ML
6- CZ FLIRTY GIRL 50ML
7- FI LADY STARK ARE
8- CZ TASTE POP 200ML
9- ES ESENS PRO SOLUTION12 104G
10- ES IMAGENES TALCO PERF 140G
338Cliente: milagrosriverav@gmail.com

 Productos recomendados:
1- LB LEDERM DIA 55+ 50ML
2- LB LEDERM OJOS 45+ 15ML
3- CZ SWEET BLACK ROLL ON 50ML
4- ES AGU SH REP MANZANILLA 750ML
5- ES AGU SH ALMENDRA 1L
6- ES OZEAN HIDROANT 75G
7- ES CFX SET TRENDY COLLECTION
8- ES SET RED & BLACK 3UN
9- LB DREAM MER 50ML
10- CZ XYF LOCION BIFASICA 90ML
339Cliente: maricela.gf.sc@gmail.com

 Productos recomendados:
1- ES NC MYSTIC GRANADA 200ML
2- CZ TRUE 50ML
3- ES NC CARIBBEAN PASSION 200ML
4- FI CRISTAL FEELING EST C+A
5- ES OXY SUP KIT30+ TSIZE CJ 3UN
6- LB BASE HYDRA ABSOLUTE 30ML
7- ES MINICHICS COL RUBI 120ML
8- ES MINICHICS COL KELLY 120ML
9- ES ESENS PRO SALON SH 350ML
10- ES OXY SUP OJOS 1

KeyboardInterrupt: 

In [61]:
columna_correo

,id_cliente,producto_1,producto_2,producto_3,producto_4,producto_5,producto_6,producto_7,producto_8,producto_9,producto_10
0,m.ureta.leon@gmail.com,210087246,200054552,200093330,200093352,200099970,200099944,200054173,200087691,200087685,200100407
1,tivisay88@hotmail.com,210088243,200064515,200083333,200091964,210088517,210088513,210090188,210090990,200087760,200046331
2,amzoason@gmail.com,200081873,200088456,200082461,200080274,200064580,200092360,200004616,200050984,P0260067000,200096219
3,asanchezc54@gmail.com,200095928,200076066,200095922,200076001,200100407,200099944,200093352,200093330,200099970,200054552
4,maria15aguilera@live.com,200046331,200092333,200101098,200087760,200091964,200083333,200064515,200098821,200051081,200075993
